In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 1. 웹드라이버 설정 및 페이지 이동
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 릴스 URL
url = "https://www.instagram.com/reel/C_IGZmRyUU9/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA=="
driver.get(url)
time.sleep(10)  # 페이지 로드 대기

# 2. 데이터 추출
# 계정명 추출
try:
    account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
except:
    account_name = "N/A"


# 좋아요 수 추출
try:
    # 좋아요 수가 포함된 span 요소를 찾습니다.
    likes_element = driver.find_element(By.XPATH, '//div[contains(text(), "likes")]')
    likes = likes_element.text.split(' ')[0]  # 'likes' 텍스트 앞의 숫자만 추출
except:
    likes = "N/A"
    

# 해시태그 추출
try:
    hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
    hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
except:
    hashtags = []

# 본문 텍스트 추출
try:
    post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
except:
    post_text = ""

# 게시 날짜 추출
try:
    post_date_element = driver.find_element(By.XPATH, '//time')
    post_date = post_date_element.get_attribute('datetime')
    post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")  # ISO 형식을 datetime으로 변환
except:
    post_date = "N/A"

# 현재 크롤링 시간
crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 3. 결과 출력 및 CSV 저장
data = {
    'account_name': account_name,
    'likes': likes,
    'hashtags': ', '.join(hashtags),
    'post_text': post_text,
    'post_date': post_date,
    'crawl_time': crawl_time,
    'post_url': url
}

# CSV 저장
df = pd.DataFrame([data])
print(df)
# df.to_csv('{}.csv'.format(account_name), index=False)
# print(f"Data saved to instagram_post_data.csv: {data}")


# 드라이버 종료
driver.quit()

   account_name likes                                           hashtags  \
0  @sh.cookkook   N/A  #입분식가정집, #송파맛집, #송리단길, #송리단길맛집, #잠실맛집, #즉석떡볶이,...   

                                           post_text            post_date  \
0  sh.cookkook\n줄서는식당에도 나온 즉떡?\n\n어릴 적부터 주구장창 먹고자...  2024-08-26 08:33:29   

            crawl_time                                           post_url  
0  2024-08-27 12:36:23  https://www.instagram.com/reel/C_IGZmRyUU9/?ut...  


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 1. 웹드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)  # 페이지 로드를 기다림

# 로그인 정보 입력 (수동 로그인을 위해 시간을 더 줌)
username = "﻿snsdkf1234@naver.com"
password = "wel1234"

driver.find_element(By.NAME, "username").send_keys(username)
driver.find_element(By.NAME, "password").send_keys(password)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(3)  # 로그인 후 대기

# 2. 특정 계정으로 이동
account_name = "seoul__nadri"  # 크롤링할 계정의 사용자 이름
driver.get(f"https://www.instagram.com/{account_name}/")
time.sleep(3)

# 3. 최근 7일간의 게시물 탐색 및 데이터 추출
posts_data = []
now = datetime.now()

In [27]:
post_elements = driver.find_elements(By.XPATH, '//article//a')
post_urls = [element.get_attribute('href') for element in post_elements]

# 데이터 저장을 위한 리스트 초기화
App = "Instagram"
data = []

# 계정 목록 (예: 사용자 계정)
accounts = ['seoul__nadri', 'sh.cookkook', 'teddy_zip_', 'hye_foodie_']

for account in accounts:
        
    for url in post_urls[0:3]: #3개만 테스트. 최대 계정당 12개만 가능
        print(url)
        driver.get(url)
        time.sleep(5)  # 페이지 로드 대기

        # 계정명 추출
        try:
            account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
        except:
            account_name = "N/A"

        # 좋아요 수 추출
        try:
            likes_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "likes")]'))
            )
            likes = likes_element.text.split(' ')[0]  # 'likes' 텍스트 앞의 숫자만 추출
        except:
            likes = "N/A"
        
        # 해시태그 추출
        try:
            hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
            hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
        except:
            hashtags = []
        
        # 본문 텍스트 추출
        try:
            post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
        except:
            post_text = ""
        
        # 게시 날짜 추출
        try:
            post_date_element = driver.find_element(By.XPATH, '//time')
            post_date = post_date_element.get_attribute('datetime')
            post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
        except:
            post_date = "N/A"
        
        # 현재 크롤링 시간
        crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        line_data = []
        line_data.append(App)
        line_data.append(account_name)
        line_data.append(likes)
        line_data.append(', '.join(hashtags))
        line_data.append(post_text)
        line_data.append(post_date)
        line_data.append(crawl_time)
        line_data.append(url)
        data.append(line_data)

# 4. 데이터프레임 생성 및 CSV 저장
df = pd.DataFrame(data, columns= ['App','Account_name', 'Likes', 'Hashtags', 'Post_text', 'Post_date', 'Crawl_time', 'Post_url'])
df.index = df.index+1
df

https://www.instagram.com/seoul__nadri/p/C_H75-ZBrd5/
https://www.instagram.com/seoul__nadri/p/C_HftwGzW5A/
https://www.instagram.com/seoul__nadri/p/C_HeAKSz27s/
https://www.instagram.com/seoul__nadri/p/C_H75-ZBrd5/
https://www.instagram.com/seoul__nadri/p/C_HftwGzW5A/
https://www.instagram.com/seoul__nadri/p/C_HeAKSz27s/
https://www.instagram.com/seoul__nadri/p/C_H75-ZBrd5/
https://www.instagram.com/seoul__nadri/p/C_HftwGzW5A/
https://www.instagram.com/seoul__nadri/p/C_HeAKSz27s/
https://www.instagram.com/seoul__nadri/p/C_H75-ZBrd5/
https://www.instagram.com/seoul__nadri/p/C_HftwGzW5A/
https://www.instagram.com/seoul__nadri/p/C_HeAKSz27s/


,App,Account_name,Likes,Hashtags,Post_text,Post_date,Crawl_time,Post_url
1,Instagram,@baguetteking_official,N/A,"#바게트킹, #수원맛집, #수원카페, #수원신상맛집, #행궁동맛집, #행궁동카페, ...",suwon_nadri\n수원 바게트킹👀\n\n드디어 행궁동에 상륙한\n수원 최초의 ...,2024-08-26 07:01:27,2024-08-27 12:37:06,https://www.instagram.com/seoul__nadri/p/C_H75...
2,Instagram,N/A,N/A,,"seoul__nadri\n화곡동 토속촌생면칼국수👀\n\n까치산역 바로 앞, 줄 서서...",2024-08-26 02:55:07,2024-08-27 12:37:22,https://www.instagram.com/seoul__nadri/p/C_Hft...
3,Instagram,@i.love.gohyang.day,N/A,"#이벤트, #상품, #경주, #추석선물, #경주데이트, #경주데이트코스, #경주가볼...",gyeongju_nadri\n경주 제2회 고향사랑의 날👀\n\n고향은 사랑을 싣고💛...,2024-08-26 02:40:09,2024-08-27 12:37:39,https://www.instagram.com/seoul__nadri/p/C_HeA...
4,Instagram,@baguetteking_official,N/A,"#바게트킹, #수원맛집, #수원카페, #수원신상맛집, #행궁동맛집, #행궁동카페, ...",suwon_nadri\n수원 바게트킹👀\n\n드디어 행궁동에 상륙한\n수원 최초의 ...,2024-08-26 07:01:27,2024-08-27 12:37:55,https://www.instagram.com/seoul__nadri/p/C_H75...
5,Instagram,N/A,N/A,,"seoul__nadri\n화곡동 토속촌생면칼국수👀\n\n까치산역 바로 앞, 줄 서서...",2024-08-26 02:55:07,2024-08-27 12:38:11,https://www.instagram.com/seoul__nadri/p/C_Hft...
6,Instagram,@i.love.gohyang.day,N/A,"#이벤트, #상품, #경주, #추석선물, #경주데이트, #경주데이트코스, #경주가볼...",gyeongju_nadri\n경주 제2회 고향사랑의 날👀\n\n고향은 사랑을 싣고💛...,2024-08-26 02:40:09,2024-08-27 12:38:28,https://www.instagram.com/seoul__nadri/p/C_HeA...
7,Instagram,@baguetteking_official,N/A,"#바게트킹, #수원맛집, #수원카페, #수원신상맛집, #행궁동맛집, #행궁동카페, ...",suwon_nadri\n수원 바게트킹👀\n\n드디어 행궁동에 상륙한\n수원 최초의 ...,2024-08-26 07:01:27,2024-08-27 12:38:45,https://www.instagram.com/seoul__nadri/p/C_H75...
8,Instagram,N/A,N/A,,"seoul__nadri\n화곡동 토속촌생면칼국수👀\n\n까치산역 바로 앞, 줄 서서...",2024-08-26 02:55:07,2024-08-27 12:39:01,https://www.instagram.com/seoul__nadri/p/C_Hft...
9,Instagram,@i.love.gohyang.day,N/A,"#이벤트, #상품, #경주, #추석선물, #경주데이트, #경주데이트코스, #경주가볼...",gyeongju_nadri\n경주 제2회 고향사랑의 날👀\n\n고향은 사랑을 싣고💛...,2024-08-26 02:40:09,2024-08-27 12:39:18,https://www.instagram.com/seoul__nadri/p/C_HeA...
10,Instagram,@baguetteking_official,N/A,"#바게트킹, #수원맛집, #수원카페, #수원신상맛집, #행궁동맛집, #행궁동카페, ...",suwon_nadri\n수원 바게트킹👀\n\n드디어 행궁동에 상륙한\n수원 최초의 ...,2024-08-26 07:01:27,2024-08-27 12:39:34,https://www.instagram.com/seoul__nadri/p/C_H75...


In [28]:
#엑셀로 저장
df.to_excel('Results/{}.xlsx'.format(account_name))
time.sleep(3)
driver.close()